<a href="https://colab.research.google.com/github/asifhasan24/Prediction-of-Fetal-Brain-Gestational-Age-Using-Multi-Head-Attention-with-Xception/blob/main/Prediction_of_Fetal_Brain_Gestational_Age_Using_Multi_Head_Attention_with_Xception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!pwd
!pip install tensorflow-addons
!kaggle datasets download -d asifhasan24/fetal-brain
!unzip /content/fetal-brain.zip

Streaming output truncated to the last 5000 lines.
  inflating: images/706/IM-2401-0023.jpg  
  inflating: images/706/IM-2401-0024.jpg  
  inflating: images/706/IM-2401-0025.jpg  
  inflating: images/706/IM-2401-0026.jpg  
  inflating: images/706/IM-2401-0027.jpg  
  inflating: images/706/IM-2401-0028.jpg  
  inflating: images/706/IM-2401-0029.jpg  
  inflating: images/706/IM-2401-0030.jpg  
  inflating: images/706/IM-2402-0001.jpg  
  inflating: images/706/IM-2402-0002.jpg  
  inflating: images/706/IM-2402-0003.jpg  
  inflating: images/706/IM-2402-0004.jpg  
  inflating: images/706/IM-2402-0005.jpg  
  inflating: images/706/IM-2402-0006.jpg  
  inflating: images/706/IM-2402-0007.jpg  
  inflating: images/706/IM-2402-0008.jpg  
  inflating: images/706/IM-2402-0009.jpg  
  inflating: images/706/IM-2402-0010.jpg  
  inflating: images/706/IM-2402-0011.jpg  
  inflating: images/706/IM-2402-0012.jpg  
  inflating: images/706/IM-2402-0013.jpg  
  inflating: images/706/IM-2402-0014.jpg  
  i

In [ ]:
import random
import cv2
import os
import numpy as np
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, MultiHeadAttention
from tensorflow.keras.models import Model
from sklearn.metrics import r2_score, mean_squared_error
from tensorflow.keras.optimizers import RMSprop,Adam
from sklearn.metrics import mean_absolute_error
import tensorflow as tf
import pandas as pd

# Load the dataset
image_folder_path = "/content/images"
img_size = 75

# Load the Excel file to get the mapping between folder names and labels
excel_file_path = '/content/images/labels.xlsx'
labels_df = pd.read_excel(excel_file_path)

# Create a dictionary that maps each folder name to its label
folder_to_label = dict(zip(labels_df['patient_id'], labels_df['ga_days']))

# Use the new lists to create training, validation, and test sets
train_img, val_img, test_img = [], [], []
train_labels, val_labels, test_labels = [], [], []

def enhance_image(image):
    # Apply Unsharp Masking
    gaussian_blur = cv2.GaussianBlur(image, (0, 0), 3)
    unsharp_image = cv2.addWeighted(image, 1.5, gaussian_blur, -0.8, 0)
    return unsharp_image

for folder_name in folder_to_label.keys():
    folder_path = os.path.join(image_folder_path, str(folder_name))
    image_files = [file for file in os.listdir(folder_path) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]

    # Shuffle the list of image files for each patient
    random.shuffle(image_files)

    # Split the list of image files for each patient into training, validation, and testing sets
    train_images, val_images, test_images = np.split(image_files, [int(0.7 * len(image_files)), int(0.8 * len(image_files))])

    for image_file in train_images:
        image_path = os.path.join(folder_path, image_file)
        image = cv2.imread(image_path)
        enhanced_image = enhance_image(image)
        train_img.append(cv2.resize(enhanced_image, (img_size, img_size)))
        train_labels.append(folder_to_label[folder_name])

    for image_file in val_images:
        image_path = os.path.join(folder_path, image_file)
        image = cv2.imread(image_path)
        enhanced_image = enhance_image(image)
        val_img.append(cv2.resize(enhanced_image, (img_size, img_size)))
        val_labels.append(folder_to_label[folder_name])

    for image_file in test_images:
        image_path = os.path.join(folder_path, image_file)
        image = cv2.imread(image_path)
        enhanced_image = enhance_image(image)
        test_img.append(cv2.resize(enhanced_image, (img_size, img_size)))
        test_labels.append(folder_to_label[folder_name])

# Convert the lists to NumPy arrays
train_img = np.array(train_img)
train_labels = np.array(train_labels)
val_img = np.array(val_img)
val_labels = np.array(val_labels)
test_img = np.array(test_img)
test_labels = np.array(test_labels)

train_img = preprocess_input(train_img)
val_img = preprocess_input(val_img)
test_img = preprocess_input(test_img)

# Load the Xception model pre-trained on ImageNet data
base_model = Xception(weights='imagenet', include_top=False, input_shape=(img_size, img_size, 3))

# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Reshape x to have sequence length dimension
x = tf.expand_dims(x, axis=1)  # Shape: (None, 1, 2048)

# Add multi-head attention layer
num_heads = 8
key_dim = 64
value_dim = 64
attention_output, attention_scores = MultiHeadAttention(num_heads=num_heads, key_dim=key_dim, value_dim=value_dim)(x, x, return_attention_scores=True)

# Flatten the attention output tensor
attention_output = tf.keras.layers.Flatten()(attention_output)

# Add a fully-connected layer with 512 hidden units and relu activation
x = Dense(512, activation='relu')(attention_output)

# Add the output layer with one neuron for regression
output_layer = Dense(1)(x)
model = Model(inputs=base_model.input, outputs=output_layer)
model.compile(loss='mean_squared_error', optimizer=Adam(lr=0.001), metrics=['mae'])

# Train the model
model.fit(train_img, train_labels, epochs=100, batch_size=16, validation_data=(val_img, val_labels))

# Evaluate on the test set
y_pred = model.predict(test_img)
r2 = r2_score(test_labels, y_pred)
mae = mean_absolute_error(test_labels, y_pred)
mse = mean_squared_error(test_labels, y_pred)
rmse = np.sqrt(mse)
print("Test Set - R-squared score: {:.3f}".format(r2))
print("Test Set - MAE: {:.3f}".format(mae))
print("Test Set - MSE: {:.3f}".format(mse))
print("Test Set - RMSE: {:.3f}".format(rmse))

# Evaluate on the validation set
val_pred = model.predict(val_img)
val_r2 = r2_score(val_labels, val_pred)
val_mae = mean_absolute_error(val_labels, val_pred)
val_mse = mean_squared_error(val_labels, val_pred)
val_rmse = np.sqrt(val_mse)
print("Validation Set - R-squared score: {:.3f}".format(val_r2))
print("Validation Set - MAE: {:.3f}".format(val_mae))
print("Validation Set - MSE: {:.3f}".format(val_mse))
print("Validation Set - RMSE: {:.3f}".format(val_rmse))
